In [ ]:
import abc
import dataclasses
import enum
import functools
import itertools
import operator
import typing

import numpy as np

In [ ]:


@dataclasses.dataclass()
class Fixture:
    home_team: typing.Hashable
    away_team: typing.Hashable
    scoreline: typing.Optional[typing.Tuple[int, int]] = None
    metadata: typing.Optional[dict] = None
       
    @property
    def home_goals(self):
        return self.scoreline[0]
       
    @property
    def away_goals(self):
        return self.scoreline[1]

In [ ]:
Fixture('Arsenal', 'Chelsea')

Fixture(home_team='Arsenal', away_team='Chelsea', scoreline=None, metadata=None)

In [ ]:
# Can this be deleted now??

class FixtureVenue(enum.Enum):
    NEUTRAL = 'Neutral venue'
    HOME_VS_AWAY = 'Home vs away'
    
    def schedule(self, team1, team2, metadata=None):
        if self == FixtureVenue.NEUTRAL:
            return [
                Fixture(home_team=team1,
                        away_team=team2,
                        metadata=metadata)
            ]
        if self == FixtureVenue.HOME_VS_AWAY:
            return [
                Fixture(home_team=team1,
                        away_team=team2,
                        metadata=metadata),
                Fixture(home_team=team2,
                        away_team=team1,
                        metadata=metadata),
            ]

In [ ]:
neutral_venue = FixtureVenue.NEUTRAL

neutral_venue.schedule('Arsenal', 'Chelsea')

[Fixture(home_team='Arsenal', away_team='Chelsea', scoreline=None, metadata=None)]

In [ ]:


class InitCompetition:
    def __init__(self, teams):
        self.teams = set(teams)

TODOs/Notes
* need some way to supply a (potentially partial) fixture/results list with `matches`
* Should we use adapters for linking to/from data, again? Could use this to just use "native" representation of a match or w/e

# Match schedulers

In [ ]:
class HomeAndAwayScheduler:
    def __init__(self, rounds=1):
        self.rounds = rounds
        
    def schedule(self, teams, metadata=None, matches=None):
        matchups = list(itertools.combinations(teams, 2))
        
        fixtures = []
        for team1, team2 in matchups:
            # TODO: need some way to merge with a list of matches
            if matches:
                pass
            
            fixtures += [
                Fixture(home_team=team1,
                        away_team=team2,
                        metadata=metadata),
                Fixture(home_team=team2,
                        away_team=team1,
                        metadata=metadata),
            ]*self.rounds
            
        return fixtures


class NeutralScheduler:
    def __init__(self, rounds=1):
        self.rounds = rounds
        
    def schedule(self, teams, metadata=None, match_data=None):
        default_metadata = {'neutral_venue': True}
        matchups = list(itertools.combinations(teams, 2))
        
        fixtures = []
        for team1, team2 in matchups:
            # TODO: need some way to merge with a list of matches
            if match_data:
                matches, adapter = match_data
                
                # If the selected match exist in the match data
                pass
            
            fixtures += [
                Fixture(home_team=team1,
                        away_team=team2,
                        metadata={**default_metadata, **(metadata or {})}),
            ]
            
        return fixtures
    

In [ ]:
teams = {'Arsenal', 'Chelsea', 'Liverpool', 'Tottenham'}

HomeAndAwayScheduler().schedule(teams)

[Fixture(home_team='Liverpool', away_team='Arsenal', scoreline=None, metadata=None),
 Fixture(home_team='Arsenal', away_team='Liverpool', scoreline=None, metadata=None),
 Fixture(home_team='Liverpool', away_team='Tottenham', scoreline=None, metadata=None),
 Fixture(home_team='Tottenham', away_team='Liverpool', scoreline=None, metadata=None),
 Fixture(home_team='Liverpool', away_team='Chelsea', scoreline=None, metadata=None),
 Fixture(home_team='Chelsea', away_team='Liverpool', scoreline=None, metadata=None),
 Fixture(home_team='Arsenal', away_team='Tottenham', scoreline=None, metadata=None),
 Fixture(home_team='Tottenham', away_team='Arsenal', scoreline=None, metadata=None),
 Fixture(home_team='Arsenal', away_team='Chelsea', scoreline=None, metadata=None),
 Fixture(home_team='Chelsea', away_team='Arsenal', scoreline=None, metadata=None),
 Fixture(home_team='Tottenham', away_team='Chelsea', scoreline=None, metadata=None),
 Fixture(home_team='Chelsea', away_team='Tottenham', scoreline=No

In [ ]:
NeutralScheduler().schedule(['Liverpool', 'Chelsea'])

[Fixture(home_team='Liverpool', away_team='Chelsea', scoreline=None, metadata={'neutral_venue': True})]

# Match Resolvers

## League table

In [ ]:


@dataclasses.dataclass()
class HeadToHead:
    def compare(self, row1, row2):
        if row1.points > row2.points:
            return -1
        elif row1.points < row2.points:
            return 1
        return 0

In [ ]:


@dataclasses.dataclass()
class LeagueTableRow:
    team: typing.Any
    goals_for: int = 0
    goals_against: int = 0
    wins: int = 0
    draws: int = 0
    losses: int = 0

    def __add__(self, row):
        if self.team != row.team:
            raise ValueError('Only TableRows with the same team can be added')

        return LeagueTableRow(
            self.team,
            self.goals_for + row.goals_for,
            self.goals_against + row.goals_against,
            self.wins + row.wins,
            self.draws + row.draws,
            self.losses + row.losses
        )

    @property
    def points(self):
        return self.wins*3 + self.draws

    @property
    def goal_difference(self):
        return self.goals_for - self.goals_against

    @staticmethod
    def from_match(team, goals_for, goals_against):
        return LeagueTableRow(
            team=team,
            goals_for=goals_for,
            goals_against=goals_against,
            wins=1 if goals_for > goals_against else 0,
            draws=1 if goals_for == goals_against else 0,
            losses=1 if goals_for < goals_against else 0
        )    

In [ ]:


class LeagueTable:
    def __init__(self, matches, tiebreak=HeadToHead()):
        # TODO: validate that rows contain only 1 of each team
        # NOTE: need to be able to parameterise the comparison function, so that we can have H2H tables
        # TODO: need to construct from matches so that can parameterise on tiebreak/h2h?
        self.matches = matches
        self.tiebreak = tiebreak
        
    def __repr__(self):
        return repr(self._rows)
    
    @property
    def _rows(self):
        """ Generate league table rows from matches. """
        match_rows = []
        for match in self.matches:
            match_rows += [
                LeagueTableRow.from_match(
                    match.home_team,
                    match.home_goals,
                    match.away_goals,
                ),
                LeagueTableRow.from_match(
                    match.away_team,
                    match.away_goals,
                    match.home_goals,
                )
            ]
        
        rows = []
        for team, team_rows in itertools.groupby(sorted(match_rows, key=lambda x: x.team), key=lambda x: x.team):
            rows.append(functools.reduce(operator.add, team_rows))
            
        return sorted(rows, key=functools.cmp_to_key(self.tiebreak.compare))
    
    @property
    def teams(self):
        return [r.teams for r in self._rows]
    
    def get_team(self, team):
        try:
            return next(x for x in self._rows if x.team == team)
        except StopIteration as err:
            raise IndexError(f'Team {repr(team)} not present in table')

    def get_position(self, index):
        if index == 0:
            IndexError(f'Position indexes start at 1, not 0. Did you mean to use index=1, instead?')
        try:
            return self._rows[
                index - 1 
                if index > 0 
                else index    # Negative indexes work the same as usual
            ]
        except IndexError:
            raise IndexError(f'Position index is out of range')

In [ ]:


class LeagueTableResolver:
    def __init__(self, tiebreak=HeadToHead()):
        self.tiebreak = tiebreak
    
    def resolve(self, matches):
        return LeagueTable(matches, tiebreak=self.tiebreak)

In [ ]:


class KnockoutResolver:
    def __init__(self):
        pass
    
    def resolve(self, matches):
        # Away goals? etc etc
        pass

In [ ]:
import json


with open('../data/premier-league-1516.json', 'r') as f:
    pl_1516 = json.load(f)
    
pl_1516[0:3]

[{'date': '2015-08-08',
  'team1': 'Manchester United FC',
  'team2': 'Tottenham Hotspur FC',
  'score': {'ft': [1, 0]}},
 {'date': '2015-08-08',
  'team1': 'AFC Bournemouth',
  'team2': 'Aston Villa FC',
  'score': {'ft': [0, 1]}},
 {'date': '2015-08-08',
  'team1': 'Leicester City FC',
  'team2': 'Sunderland AFC',
  'score': {'ft': [4, 2]}}]

In [ ]:
LeagueTableResolver().resolve(matches=[
    Fixture(p['team1'], p['team2'], scoreline=p['score']['ft']) for p in pl_1516
])._rows

[LeagueTableRow(team='Leicester City FC', goals_for=68, goals_against=36, wins=23, draws=12, losses=3),
 LeagueTableRow(team='Arsenal FC', goals_for=65, goals_against=36, wins=20, draws=11, losses=7),
 LeagueTableRow(team='Tottenham Hotspur FC', goals_for=69, goals_against=35, wins=19, draws=13, losses=6),
 LeagueTableRow(team='Manchester City FC', goals_for=71, goals_against=41, wins=19, draws=9, losses=10),
 LeagueTableRow(team='Manchester United FC', goals_for=49, goals_against=35, wins=19, draws=9, losses=10),
 LeagueTableRow(team='Southampton FC', goals_for=59, goals_against=41, wins=18, draws=9, losses=11),
 LeagueTableRow(team='West Ham United FC', goals_for=65, goals_against=51, wins=16, draws=14, losses=8),
 LeagueTableRow(team='Liverpool FC', goals_for=63, goals_against=50, wins=16, draws=12, losses=10),
 LeagueTableRow(team='Stoke City FC', goals_for=41, goals_against=55, wins=14, draws=9, losses=15),
 LeagueTableRow(team='Chelsea FC', goals_for=59, goals_against=53, wins=12

# Competition blocks

note: need to have basic blocks and composite blocks with the same api

In [ ]:
class SequentialCompetition:
    pass


class CompositeCompetition:
    pass

In [ ]:
efl_championship = {
    'rounds': {
        'main': (HomeAndAwayScheduler(), LeagueTableResolver()),
        'playoff_1': (HomeAndAwayScheduler(), KnockoutResolver()),
        'playoff_2': (HomeAndAwayScheduler(), KnockoutResolver()),
        'final': (NeutralScheduler(), KnockoutResolver()),
    },
    'links': [
        {'from': 'main', 'to': 'playoff_1', ...}  # How do we say _which_ teams? -> need to either add a filtering blocksd, or add it into the edge definition
    ]
}

# Composing competition blocks

# Competition blocks

In [ ]:
class CompetitionBlockABC(abc.ABC):
    def __init__(self, scheduler=HomeAndAwayScheduler()):
        self.scheduler = scheduler
    
    def predict(self, teams, fn, metadata=None, matches=None):
        """ Predict the matches according to the scheduler. """
        fixtures = self.scheduler.schedule(teams, metadata, matches)
        for match in fixtures:
            if match.scoreline:
                continue
            
            match.scoreline = fn(match)
        return fixtures
    
    @abc.abstractmethod
    def resolve(self, matches):
        """ Take played/predicted matches and resolve them according to the block logic. """
        pass

In [ ]:


class AllPlayAll:
    """ ... """
    
    def __init__(self, scheduler=HomeAndAwayScheduler()):
        self.scheduler = scheduler
    
    def predict(self, teams, fn, metadata=None, matches=None):
        fixtures = self.scheduler.schedule(teams, metadata, matches)
        for match in fixtures:
            if match.scoreline:
                continue
            
            match.scoreline = fn(match)
        return fixtures
    
    def resolve()

In [ ]:
teams = {'Arsenal', 'Chelsea', 'Liverpool', 'Tottenham'}

mini_league = AllPlayAll()
mini_league.predict(teams, lambda x: (1, 0))

In [ ]:
mini_league.predict(lambda x: (2, 1))

In [ ]:


class Knockout:
    def __init__(self, teams, matches_per_team = FixtureVenue.NEUTRAL, matches=[]):
        n_teams = len(teams)
        if (n_teams > 1) and (n_teams % 2 != 0):
            raise ValueError('A knockout block must have an even number of teams!')
        
        self.teams = teams
        self.matches_per_team = matches_per_team
        self.matches = matches
        
        # NOTE: in group-stage -> knockout tournaments, how do you ensure the correct joining? Just do multiple
        # `Knockout`s?
        
    def schedule(self):
        pass

## "Glue" competition blocks

In [ ]:


class Split:
    def __init__(self, previous):
        # NOTE: You can only split an AllPlayAll table, I think?
        pass

In [ ]:


class Combine:
    pass

## Composite competition block

In [ ]:


class Competition:
    pass

    # TODO: some kind of graphvis plotting method